# Модель оценки заголовка

Введенные или сгенерированные заголовки нужно как-то оценивать. Так как главной целью обычно является увеличение количества просмотров, то в качестве критерия нужно использовать число просмотров у ранее опубликованных статей. То есть перед нами стоит задача **регрессии**: на входе заголовок, на выходе число (балл от 0.0 до 10.0 с точностью 0.1).


In [1]:
 import os
 os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 1. Подготовка данных для модели оценки заголовка



In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
data_path='/content/gdrive/My Drive/Colab Notebooks/title/data'

Mounted at /content/gdrive


In [3]:
Xy = pd.read_pickle(f'{data_path}/Xy.pickle', compression='gzip')
X, y = Xy.title, Xy.score
del Xy

Разобьем данные на обучающую, тестовую и валидационную выборки. Валидационную выборку мы используем для оценки и настройки, не трогая тестовую выборку, оставляя ее для оценки лишь конечной модели.

In [4]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(X.to_list(), y.to_list(), test_size=.2)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

# 2. Построение модели глубокого обучения

Для работы с текстами мы используем библиотеку [transformers](https://huggingface.co/transformers/), обладающую высокой эффективностью для задач распознавания особенностей текста (NLU) и его генерации (NLG). Библиотека предоставляет удобный интерфейс для работы с предобученными NLP-моделями на основе архитектуры transformer. Фактически это pytorch-модели для NLP-задач, которые легко переводить в tensorflow-модели и обратно.


In [5]:
%%bash
pip3 install transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=e5061a28dc2ef324602284ca80403116a76d2bee94f487dcd5a3a3c9ce164c59
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [6]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

model_name = 'DeepPavlov/rubert-base-cased'
model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

# в случае ошибки, учесть fast-версию
# https://fantashit.com/autotokenizer-from-pretrained-bert-throws-typeerror-when-encoding-certain-input/
tokenizer = BertTokenizer.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(119547, 768, padding_idx=0)

Передадим тексты токенизатору. Флаги `truncation = True` и `padding = True` гарантируют, что последовательности будут дополнены до одинаковой длины и усечены, чтобы не превышать максимальную входную длину последовательности.

In [7]:
max_length = 32  # макс. длина заголовка в токенах
train_encodings = tokenizer(train_texts,
                            truncation=True,
                            padding=True,
                            max_length=max_length)

val_encodings = tokenizer(val_texts,
                          truncation=True,
                          padding=True,
                          max_length=max_length)

test_encodings = tokenizer(test_texts,
                           truncation=True,
                           padding=True,
                           max_length=max_length)

Теперь представим размеченные тексты и метки в виде `Dataset`-объекта. Для этого наследуем класс от `torch.utils.data.Dataset`, в котором реализуем методы `__len__` и `__getitem__`. Это позволяет отправлять данные пакетно, батчами и обучать модель с помощью метода `forward()`.

In [8]:
import torch

class TitlesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    

train_dataset = TitlesDataset(train_encodings, train_labels)
val_dataset = TitlesDataset(val_encodings, val_labels)
test_dataset = TitlesDataset(test_encodings, test_labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

Step,Training Loss
10,0.476100
20,0.615400
30,0.519900
40,0.668100
50,0.644900
60,0.457300
70,0.546500
80,0.569800
90,0.613700
100,0.534600


In [ ]:
# from torch.utils.data import DataLoader
# from transformers import BertForSequenceClassification, AdamW

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# torch.cuda.empty_cache()

# model_name = 'bert-base-multilingual-cased'
# model = BertForSequenceClassification.from_pretrained(model_name,
#                                                       num_labels = 101,
#                                                       output_attentions = False,
#                                                       output_hidden_states = False)
# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset,
#                           batch_size=8,
#                           shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids,
#                         attention_mask=attention_mask,
#                         labels=labels)
#         loss = outputs[0]
#         print(loss)
#         loss.backward()
#         optim.step()

# model.eval()